## Model 2

In [124]:
!pip install pathway bokeh --quiet

In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [126]:
df = pd.read_csv("https://raw.githubusercontent.com/Awani-Soni/dynamic-pricing-capstone-sa/main/dataset.csv")
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [127]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Encode traffic levels
traffic_map = {"low": 0.1, "average": 0.5, "high": 1.0}
df["TrafficEncoded"] = df["TrafficConditionNearby"].map(traffic_map)

# Encode vehicle types
vehicle_map = {"car": 0.7, "bike": 0.4, "truck": 1, "cycle": 0.1}
df["VehicleEncoded"] = df["VehicleType"].str.lower().map(vehicle_map)


# Sort the DataFrame by the new 'Timestamp' column and unique SystemCodeNumber
# and reset the index
df = df.sort_values(['Timestamp', 'SystemCodeNumber']).reset_index(drop=True)

In [128]:
# Select the relevant and encoded columns
df_cleaned = df[[
    'Timestamp',
    'SystemCodeNumber',
    'Occupancy',
    'Capacity',
    'QueueLength',
    'TrafficEncoded',
    'IsSpecialDay',
    'VehicleEncoded',
    'Latitude',
    'Longitude'
]]

# Save to CSV for streaming
df_cleaned.to_csv("parking_stream_2.csv", index=False)

In [129]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    Timestamp: str               # Timestamp of the observation
    SystemCodeNumber: str        # Unique ID of the parking lot
    Occupancy: int               # Current number of parked vehicles
    Capacity: int                # Total capacity of the parking lot
    QueueLength: int             # Number of vehicles waiting to enter
    TrafficEncoded: float        # Encoded traffic condition (low=0.1, medium=0.5, high=1.0)
    IsSpecialDay: int            # 1 = special day/holiday, 0 = normal day
    VehicleEncoded: float        # Encoded vehicle type (car=0.7, bike=0.4, truck=1, cycle=0.1)
    Latitude: float              # Latitude of the parking lot
    Longitude: float             # Longitude of the parking lot

In [130]:
# Load the dataset as a simulated real-time stream using Pathway
# input_rate=14 means 14 rows will be streamed per second
# Since the CSV is sorted by ['Timestamp', 'SystemCodeNumber'], this simulates
# data for all 14 parking lots at the same timestamp (every 30 minutes)
data = pw.demo.replay_csv("parking_stream_2.csv", schema=ParkingSchema, input_rate=100)

In [131]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new column to the data stream
# - 't' contains the parsed full datetime
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt)  # Full datetime for each row
)

###  Model 2: Demand-Based Dynamic Price Model

This model introduces a more flexible, data-driven approach to pricing by incorporating multiple real-time features. Instead of only using occupancy rate, the model constructs a **demand score** that reflects multiple factors influencing parking demand.

---

####  Demand Function:

The price is computed based on the following formula:

$$
\text{Price}_t = \text{BasePrice} \times \left(1 + \lambda \cdot \text{NormalizedDemand} \right)
$$

Where:

- **BasePrice** = ₹10
- **NormalizedDemand** is computed from:

$$
\text{Demand} = \alpha \cdot \frac{\text{Occupancy}}{\text{Capacity}} + \beta \cdot \text{QueueLength} - \gamma \cdot \text{TrafficLevel} + \delta \cdot \text{IsSpecialDay} + \varepsilon \cdot \text{VehicleType}
$$

Then demand is normalized:

$$
\text{NormalizedDemand} = \min\left( \max\left(\frac{\text{Demand}}{10}, 0 \right), 1 \right)
$$

Finally, price is bounded:

$$
\text{Price}_t = \min\left( \max\left(10 \cdot (1 + \lambda \cdot \text{NormalizedDemand}), 5 \right), 20 \right)
$$

---

#### Model Parameters:

- \( $\alpha$ = 0.4 \): Sensitivity to occupancy
- \( $\beta$ = 0.3 \): Influence of queue length
- \( $\gamma$ = 0.2 \): Penalty for high traffic
- \( $\delta$ = 0.2 \): Boost for special days
- \( $\varepsilon$ = 0.5 \): Weight for vehicle type
- \( $\lambda$ = 1.0 \): Overall demand scaling factor

---

#### Session Window:

- A **0.3-hour session window** is applied per parking lot.
- For each session, features are aggregated using `max()`, as each lot contributes one record per session.

---

#### Summary:

- This model captures **demand elasticity** by considering real-world contextual factors.
- Prices dynamically fluctuate between ₹5 and ₹20 depending on demand.
- The approach ensures smoother pricing while remaining bounded and explainable.


In [132]:
from datetime import timedelta

# Model 2 coefficients
α = 0.4   # Occupancy rate
β = 0.3   # Queue length
γ = 0.2   # Traffic level (already encoded)
δ = 0.2   # Special day
ε = 0.5   # Vehicle type weight (already encoded)
λ = 1.0   # Demand scaling

# Pathway Model 2 Pipeline
model2_window = (
    data.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.session(max_gap=timedelta(hours=0.3)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        SystemCodeNumber = pw.this._pw_instance,
        Occupancy = pw.reducers.max(pw.this.Occupancy),
        Capacity = pw.reducers.max(pw.this.Capacity),
        QueueLength = pw.reducers.max(pw.this.QueueLength),
        TrafficEncoded = pw.reducers.max(pw.this.TrafficEncoded),  # encoded int
        IsSpecialDay = pw.reducers.max(pw.this.IsSpecialDay),
        VehicleEncoded = pw.reducers.max(pw.this.VehicleEncoded)  # encoded int
    )
    .with_columns(
        Price_Model_2 = pw.apply(lambda nd: round(min(max(10 * (1 + λ * nd), 5), 20), 2), (pw.apply(lambda d: min(max(d / 10, 0), 1), (α * (pw.this.Occupancy / pw.this.Capacity) + β * pw.this.QueueLength - γ * pw.this.TrafficEncoded + δ * pw.this.IsSpecialDay + ε * pw.this.VehicleEncoded))))
    )
)

### Model 2 Plot: Price Over Time

The following plot shows dynamic prices over time for three selected parking lots (**BHMBCCMKT01**, **Broad Street**, and **Others-CCCPS119a**) using **Model 2**. Each line represents the price trend for one lot, calculated based on real-time demand factors like occupancy, queue length, traffic, and vehicle type.


In [140]:
import bokeh.plotting
import panel as pn
from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource

pn.extension()

# Select 2 lots to display
lots = ["BHMBCCMKT01", "Broad Street", "Others-CCCPS119a"]
# Define a list of 2 colors manually
colors = ["#1f77b4", "#ff7f0e", "#ff77b5"] # Example colors from Category10[3]


# Step 1: Filter model1_window for each lot
filtered_tables = [
    model2_window.filter(pw.this.SystemCodeNumber == lot)
    for lot in lots
]

# Step 2: Helper function that returns a full figure
def create_figure(src, lot_name, color):
    fig = bokeh.plotting.figure(
        height=300,
        width=400,
        title=f"{lot_name} - Price Over Time",
        x_axis_type="datetime",
        y_axis_label="Price ($)",
    )
    fig.line("t", "Price_Model_2", source=src, line_width=2, color=color)
    return fig

# Step 3: Use Panel Row to show both plots
viz = pn.Row(
    *[
        table.plot(
            lambda src, lot=lot, color=color: create_figure(src, lot, color),
            sorting_col="t"
        )
        for table, lot, color in zip(filtered_tables, lots, colors)
    ]
)

viz

Row
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [141]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()